In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers


data = pd.read_csv('D:/GP/datasets/mentalevaluation_updated.csv')


X = data.drop(['PTSD', 'Bipolar disorder', 'Anxiety disorder', 'Depression', 'Schizophrenia', 'No mental illness detected'], axis=1)
y = data[['PTSD', 'Bipolar disorder', 'Anxiety disorder', 'Depression', 'Schizophrenia', 'No mental illness detected']]


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

X_processed = preprocessor.fit_transform(X)
y_encoded = y.values  


X_train, X_test, y_train, y_test = train_test_split(X_processed, y_encoded, test_size=0.1, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))),
    Dropout(0.3),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.05)),
    Dense(6, activation='sigmoid')  # Changed to 6 neurons
])


optimizer = Adam(learning_rate=0.0065)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)


history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1, 
                    callbacks=[early_stopping])


y_pred = model.predict(X_test)


y_pred_classes = (y_pred > 0.5).astype(int)


overall_accuracy = accuracy_score(y_test, y_pred_classes)

print(f"Overall Accuracy: {overall_accuracy:.4f}")


for i, label in enumerate(y.columns):
    print(f"\nClassification Report for {label}:\n", classification_report(y_test[:, i], y_pred_classes[:, i]))


for i, label in enumerate(y.columns):
    print(f"\nConfusion Matrix for {label}:\n", confusion_matrix(y_test[:, i], y_pred_classes[:, i]))


X_train shape: (4889, 53)
X_test shape: (544, 53)
Epoch 1/50


D:\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.0841 - loss: 1.3799 - val_accuracy: 0.1534 - val_loss: 0.2512
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3642 - loss: 0.2237 - val_accuracy: 0.5297 - val_loss: 0.1704
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6669 - loss: 0.1655 - val_accuracy: 0.7669 - val_loss: 0.1584
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7442 - loss: 0.1490 - val_accuracy: 0.9346 - val_loss: 0.1415
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7712 - loss: 0.1442 - val_accuracy: 0.9284 - val_loss: 0.1371
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7866 - loss: 0.1356 - val_accuracy: 0.8855 - val_loss: 0.1488
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8167 - loss: 0.1394 - val_accuracy: 0.7239 - val_loss: 0.1312
Epoch 8/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7715 - loss: 0.1290 - val_accuracy: 0.6994 - val_loss: 0.1296
E

D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: